# *Brain Tumor Radiogenomic Classification* 
#### -> _3D CNN with PyTorch_

### Problem Description 
A malignant tumor in the brain is a life-threatening condition. Known as glioblastoma, it's both the most common form of brain cancer in adults and the one with the worst prognosis, with median survival being less than a year. The presence of a specific genetic sequence in the tumor known as MGMT promoter methylation has been shown to be a favorable prognostic factor and a strong predictor of responsiveness to chemotherapy. 

### Data Description

U.Baid, et al., “The RSNA-ASNR-MICCAI BraTS 2021 Benchmark on Brain Tumor Segmentation and Radiogenomic Classification”, arXiv:2107.02314, 2021.

### Outline
1. [Exploring the data](#exploring)

In [12]:
# !pip install pydicom

In [7]:
# necessary imports
import os
import sys
import glob
from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from time import time
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms, utils

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [10]:
# sagemaker specific imports
import boto3
import sagemaker

# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

sagemaker_bucket = sagemaker_session.default_bucket()
print(sagemaker_bucket)

sagemaker-us-west-2-904606187431


<a id='exploring'></a>
### Exploring the Data

In [11]:
# project specific data paths
train_labels_key = 'train_labels.csv'

s3_prefix = 'Radiogenomic'
output_path = 's3://{}/{}/'.format(sagemaker_bucket, s3_prefix)
print('Model artifacts and data will be stored in ', output_path)

train_labels = pd.read_csv(os.path.join(output_path, train_labels_key))
print("Num train labels: ", len(train_labels))
train_labels.head()

Model artifacts and data will be stored in  s3://sagemaker-us-west-2-904606187431/Radiogenomic/
Num train labels:  585


,BraTS21ID,MGMT_value
0,0,1
1,2,1
2,3,0
3,5,1
4,6,1
